In [2]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from collections import OrderedDict
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from PIL import Image
import os
from torch.optim import lr_scheduler
import copy

In [3]:
from torch.nn.modules import batchnorm
bs = 1
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.1)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.2)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.3)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size=4,padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Dropout2d(0.3)
        )
        self.conv8 = nn.Sequential(
            nn.Conv2d(256,256,kernel_size=3,padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )

        self.fc = nn.Linear(256, 7)  # number of classes =7

    def forward(self, x):
        x = self.conv8(self.conv7(self.conv6(self.conv5(self.conv4(self.conv3(self.conv2(self.conv1(x))))))))        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [4]:

data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomCrop((256, 256), pad_if_needed=True),
        transforms.RandomAffine(10),
        transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5),
#         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
#         transforms.RandomGrayscale(p=0.5),
        transforms.RandomRotation(10),
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

}
data_dir='/tmp/images/images'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}
dataloders={x: torch.utils.data.DataLoader(image_datasets[x],batch_size=64,shuffle=True,num_workers=5)
            for x in['train','validation']}

dataset_size={x:len(image_datasets[x]) for x in ['train','validation']}
class_names= image_datasets['train'].classes

print(class_names)
print(len(class_names))

#if torch.cuda.is_available():
#  device=device.cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
7


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomCrop((256, 256), pad_if_needed=True),
        transforms.RandomAffine(10),
        transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5),
#         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
#         transforms.RandomGrayscale(p=0.5),
        transforms.RandomRotation(10),
        transforms.Resize((256,256)),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/tmp/images/images'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                              shuffle =True,
                                              num_workers=10)
              for x in ['train', 'validation']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes

print(class_names)
print(len(class_names))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
7


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Facial_Expression_Recog/archive.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()



In [6]:
print('Test set dogs' , len(os.listdir('/tmp/images/images/train/angry')))
print('Test set cats' , len(os.listdir('/tmp/images/images/train/disgust')))
print('Train set dogs' , len(os.listdir('/tmp/images/images/validation/angry')))
print('Train set cats' , len(os.listdir('/tmp/images/images/validation/disgust')))

Test set dogs 3993
Test set cats 436
Train set dogs 960
Train set cats 111


In [7]:
def train(model, criterion, optimizer, num_epoch=25):
  best_model=copy.deepcopy(model.state_dict())
  best_Acc=0.0

  for epoch in range(num_epoch):
    print('Epoch {}/{}'.format(epoch +1 , num_epoch))

    for phase in ['train','validation']:
      if phase=='train':
        model.train()
      else:
        model.test()
      
      running_loss = 0.0
      running_corrects = 0

      with tqdm(dataloders[phase],unit="batch") as tepoch:
        for inputs, labels in tepoch:
          #tepoch.set_decription(f"Epoch{epoch}")
          input=inputs.to(device)
          labels=labels.to(device)

          optimizer.zero_grad()

          with torch.set_grad_enabled(phase=='train'):
            outputs=model(inputs)
            _, preds=torch.max(outputs,1)
            loss=criterion(outputs,labels)

            if phase=='train':
              loss.backward()
              optimizer.step()
          
          running_loss += loss.item()*inputs.size(0)
          running_corrects += torch.sum(preds==labels.data)
      
      
      epoch_loss=running_loss/dataset_sizes[phase]
      epoch_acc=running_corrects.double()/dataset_sizes[phase]
      
      print('{} Loss : {:.4f} Acc : {:.4f}').format(phase, epoch_loss,epoch_acc)
      
      if epoch_acc > best:
            best = epoch_acc
            torch.save(model.state_dict(), "best_2.ckpt")
      
      print()

  print('Best val Acc: {:4f}'.format(best))

  model.load_state_dict(best_model)
  return model

In [8]:
model_ft=NeuralNet()
model_ft=model_ft.to(device)

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()

#optimizer_ft=optim.GSD(model_ft.parameters(),lr=0.001)
optimizer_ft=Adam(model_ft.parameters(), lr=0.001, weight_decay=1e-4)

#exp_lr_scheduler=lr_schedular.StepLR(optimizer_fr, step_size=7, gamma=0.1)

In [ ]:
model_ft = train(model_ft, criterion, optimizer_ft, num_epoch=15)